In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
coupon_list_train = pd.read_csv('../input/coupon_list_train.csv')
coupon_list_test = pd.read_csv('../input/coupon_list_test.csv')
user_list = pd.read_csv('../input/user_list.csv')
coupon_purchases_train = pd.read_csv("../input/coupon_detail_train.csv")

In [ ]:
set(coupon_list_test.GENRE_NAME.values)

In [ ]:
print (user_list.shape, coupon_list_test.shape, coupon_list_train.shape, coupon_purchases_train.shape)

In [ ]:
len(np.unique(coupon_purchases_train.COUPON_ID_hash.values))

In [ ]:
len(np.unique(coupon_list_train.COUPON_ID_hash.values))

In [ ]:
sum([x in np.unique(coupon_list_train.COUPON_ID_hash.values) for x in np.unique(coupon_purchases_train.COUPON_ID_hash.values)])

In [ ]:
set(purchased_coupons_train.COUPON_ID_hash.values).issubset(set(coupon_list_train.COUPON_ID_hash.values))

In [ ]:
user_list.info(null_counts=True)

In [ ]:
### merge to obtain (USER_ID) <-> (COUPON_ID with features) training set
purchased_coupons_train = coupon_purchases_train.merge(coupon_list_train,
                                                 on='COUPON_ID_hash',
                                                 how='inner')

In [ ]:
purchased_coupons_train.info(null_counts=True)

In [ ]:
features = ['COUPON_ID_hash', 'USER_ID_hash',
            'GENRE_NAME', 'DISCOUNT_PRICE', 'PRICE_RATE',
            'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU',
            'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY',
            'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name', 'small_area_name']
purchased_coupons_train = purchased_coupons_train[features]

In [ ]:
purchased_coupons_train.info(null_counts=True)

In [ ]:
### create 'dummyuser' records in order to merge training and testing sets in one
coupon_list_test['USER_ID_hash'] = 'dummyuser'

In [ ]:
### filter testing set consistently with training set
coupon_list_test = coupon_list_test[features]

In [ ]:
### merge sets together
combined = pd.concat([purchased_coupons_train, coupon_list_test], axis=0)

In [ ]:
print (purchased_coupons_train.shape, coupon_list_test.shape, combined.shape)

In [ ]:
168996 + 310

In [ ]:
### create two new features
combined['DISCOUNT_PRICE'] = 1 / np.log10(combined['DISCOUNT_PRICE'])
combined['PRICE_RATE'] = (combined['PRICE_RATE'] / 100) ** 2
features.extend(['DISCOUNT_PRICE', 'PRICE_RATE'])

In [ ]:
### convert categoricals to OneHotEncoder form
categoricals = ['GENRE_NAME', 'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED',
                'USABLE_DATE_THU', 'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN',
                'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name', 'small_area_name']
combined_categoricals = combined[categoricals]
combined_categoricals = pd.get_dummies(combined_categoricals,
                                    dummy_na=False)

In [ ]:
### leaving continuous features as is, obtain transformed dataset
continuous = list(set(features) - set(categoricals))
combined = pd.concat([combined[continuous], combined_categoricals], axis=1)

In [ ]:
### remove NaN values
NAN_SUBSTITUTION_VALUE = 1
combined = combined.fillna(NAN_SUBSTITUTION_VALUE)

In [ ]:
### split back into training and testing sets
train = combined[combined['USER_ID_hash'] != 'dummyuser']
test = combined[combined['USER_ID_hash'] == 'dummyuser']
test.drop('USER_ID_hash', inplace=True, axis=1)

In [ ]:
### find most appropriate coupon for every user (mean of all purchased coupons), in other words, user profile
train_dropped_coupons = train.drop('COUPON_ID_hash', axis=1)
user_profiles = train_dropped_coupons.groupby(by='USER_ID_hash').apply(np.mean)

### creating weight matrix for features
FEATURE_WEIGHTS = {
    'GENRE_NAME': 2,
    'DISCOUNT_PRICE': 2,
    'PRICE_RATE': 0,
    'USABLE_DATE_': 0,
    'large_area_name': 0.5,
    'ken_name': 1,
    'small_area_name': 5
}

In [ ]:
train_dropped_coupons.info(null_counts=True)

In [ ]:
user_profiles.info(null_counts=True)

In [ ]:
def find_appropriate_weight(weights_dict, colname):
    for col, weight in weights_dict.items():
        if col in colname:
            return weight
    raise ValueError

In [ ]:
W_values = [find_appropriate_weight(FEATURE_WEIGHTS, colname)
            for colname in user_profiles.columns]
W = np.diag(W_values)

In [ ]:
### find weighted dot product(modified cosine similarity) between each test coupon and user profiles
test_only_features = test.drop('COUPON_ID_hash', axis=1)
similarity_scores = np.dot(np.dot(user_profiles, W),
                           test_only_features.T)

In [ ]:
### create (USED_ID)x(COUPON_ID) dataframe, similarity scores as values
coupons_ids = test['COUPON_ID_hash']
index = user_profiles.index
columns = [coupons_ids[i] for i in range(0, similarity_scores.shape[1])]
result_df = pd.DataFrame(index=index, columns=columns,
                      data=similarity_scores)

In [ ]:
def get_top10_coupon_hashes_string(row):
    row.sort()
    return ' '.join(row.index[-10:][::-1].tolist())

output = result_df.apply(get_top10_coupon_hashes_string, axis=1)


output_df = pd.DataFrame(data={'USER_ID_hash': output.index,
                               'PURCHASED_COUPONS': output.values})
output_df_all_users = pd.merge(user_list, output_df, how='left', on='USER_ID_hash')

In [ ]:
output_df_all_users.to_csv('cosine_sim_python.csv', header=True,
                           index=False, columns=['USER_ID_hash', 'PURCHASED_COUPONS'])

In [ ]:
output_df_all_users.head()

In [ ]:
user_profiles

In [ ]:
user_profiles.columns